<a href="https://colab.research.google.com/github/Varun-626/flask-hello-world/blob/master/Project_SPIS_Input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''This code cleans up the tweets_01-08-2021.csv file and saves the 'text', 'retweets' & 'date' columes to a new .csv file'''

# 'text', 'isRetweet', 'isDeleted', 'favorites', 'retweets', 'date', 'device'

import pandas as pd
import re

def remove_https_links_from_csv(input_filepath, output_filepath):
        with open(input_filepath, 'r') as infile, open(output_filepath, 'w') as outfile:
            for line in infile:
                # Use regex to find and replace HTTPS links
                # This regex matches http or https followed by anything until a space or end of line
                cleaned_line = re.sub(r'https?://\S+', '', line)
                outfile.write(cleaned_line)

# Load the original CSV
df = pd.read_csv('tweets_01-08-2021.csv')

# Remove rows where the 'device' column is 'TweetDeck'
df_filtered = df[df['device'] != 'TweetDeck']

# Remove rows where the 'isRetweet' column is 't' (true)
df_filtered = df[df['isRetweet'] != 't']

# Remove rows where the 'isDeleted' column is 't' (true)
df_filtered = df[df['isDeleted'] != 't']

# Drop unwanted columns
columns_to_remove = ['id', 'device', 'favorites', 'isFlagged', 'isDeleted', 'isRetweet']
df_new = df.drop(columns=columns_to_remove)

# Words to remove
words_to_remove = ['RT']

# Remove words from 'text' column
df['text'] = df['text'].replace(
    to_replace=r'\b(?:' + '|'.join(words_to_remove) + r')\b',
    value='',
    regex=True
)

# Regex pattern to match words starting with @ or # or http(s)
pattern_at = r'\s*@\w+'
pattern_hash = r'\s*#\w+'
pattern_https = r'https:\/\/\S+'

# Apply the pattern to the column
df['text'] = df['text'].str.replace(pattern_at, '', regex=True).str.strip()
df['text'] = df['text'].str.replace(pattern_hash, '', regex=True).str.strip()
df['text'] = df['text'].str.replace(pattern_https, '', regex=True).str.strip()

# Strip extra whitespace
df['text'] = df['text'].str.strip()

# Drop rows where 'text' column is empty
df = df.dropna(subset=['text'])


remove_https_links_from_csv('tweets_01-08-2021.csv', 'filteredTweets.csv')

# Save to a new CSV file
df.to_csv('filteredTweets.csv', index=False)

print("Saved to filteredTweets.csv")

Saved to filteredTweets.csv


In [ ]:
'''This code takes the filteredTweets.csv file and saves it as trumpTweets.txt in an easy to read format'''

import pandas as pd

# Load the CSV file
df = pd.read_csv('filteredTweets.csv')

# Drop rows with missing values in either column
df = df.dropna(subset=['text', 'date'])

# Convert everything to string (safe for numbers)
df['text'] = df['text'].apply(str)

#df['date'] = df['date'].apply(str)

# Combine columns ("text: date")
combined = df['text'] #+ ': ' + df['date']

# Combine into one text block (each tweet on its own line)
text = "\n".join(combined)

# Save to TXT file
with open('trumpTweets.txt', 'w', encoding='utf-8') as f:
    f.write(text)

# Read the trumpTweets.txt file
with open('trumpTweets.txt', 'r', encoding='utf-8') as infile:
    lines = infile.readlines()

# Filter out empty or whitespace-only lines
cleaned_lines = [line for line in lines if line.strip() != '']

# Overwrite the original
with open('trumpTweets.txt', 'w', encoding='utf-8') as outfile:
    outfile.writelines(cleaned_lines)

print("Saved to trumpTweets.txt")

Saved to trumpTweets.txt


In [21]:
from datetime import datetime
import csv
import re

t = []

with open('tweets_01-08-2021.csv', 'r') as csv_file:

  csv_reader = csv.DictReader(csv_file)
  for row in csv_reader:
    match = re.search(r'\b\d{2}:\d{2}:\d{2}\b', row['date'])
    time_string = match.group() if match else ''
    t.append(time_string)

def formatdate(date):

    dateFormat = '%Y-%m-%d %H:%M:%S'
    d = datetime.strptime(date, dateFormat)
    return d.strftime('%m/%d/%Y')

listTweet = []
USDDollarIndex = []
SPDollarIndex = []

def datesort(date):
  data = date

  sorted_data_desc = sorted(data, key=lambda item: datetime.strptime(item[0], "%m/%d/%Y"), reverse=True)


DollarIndexChangeFormatted = []

output_file = 'DataInputwTimeStamp.csv'

with open('tweets_01-08-2021.csv', 'r', newline='') as filteredTweets_csv:
    reader = csv.DictReader(filteredTweets_csv)

    for row in reader:
        textValue = row['text']
        dateValue = row['date']
        dateValue = formatdate(dateValue)
        listTweet.append([dateValue, textValue])

with open('US Dollar Index Historical Data.csv', 'r', newline='') as USDIHD_csv:
    reader = csv.DictReader(USDIHD_csv)

    #print(reader.fieldnames) for debugging

    for row in reader:
        changeValue = row['Change %']
        dateValue = row['\ufeff"Date"']
        USDDollarIndex.append([dateValue, changeValue])

with open('S&P 500 Historical Data (1).csv', 'r', newline='') as SP_csv:
    reader = csv.DictReader(SP_csv)

    print(reader.fieldnames)

    for row in reader:
        changeValue = row['Change %']
        dateValue = row['\ufeff"Date"']
        SPDollarIndex.append([dateValue, changeValue])

# Convert into a dictionary for faster lookup
dictDI = {row[0]: row[1:] for row in USDDollarIndex}
dictSP = {row[0]: row[1:] for row in SPDollarIndex}

# Combine based on matching dates
combined = []

for row in listTweet:
    date = row[0]
    if date in dictDI and date in dictSP:
        combined.append([date]+ row[1:] + dictDI[date] + dictSP[date])

saver = []
for x in range(len(combined)):
    combined[x][2] = combined[x][2].replace('%', '')
    combined[x][2] = float(combined[x][2])
    combined[x][3] = combined[x][3].replace('%', '')
    combined[x][3] = float(combined[x][3])
    combined[x].append(t[x])
    if len(combined[x][1]) < 5:
      saver.append(x)

for x in saver:
  try :
      combined.remove(combined[x])
  except:
      pass

#datesort(combined)
#print(combined)

import pandas as pd
import numpy as np

df = pd.DataFrame(combined, columns=['Date', 'Tweet', 'USDChange%', 'SPChange%', 'Timestamp'])

df['Tweet'].replace('', np.nan, inplace=True)
df['Tweet'].replace(':', np.nan, inplace=True)

df_cleaned = df.dropna()

df.to_csv(output_file, index=False)

file_path = output_file
column_to_check = 'Tweet'

with open(file_path, 'r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    rows = [row for row in reader if row[column_to_check] and row[column_to_check].strip()]
    fieldnames = reader.fieldnames

with open(file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

df = pd.read_csv('DataInputwTimeStamp.csv')

df['Date'] = pd.to_datetime(df['Date'])
sorted_df = df.sort_values(by='Date', ascending=True)
sorted_df.to_csv('DataInputwTimeStamp.csv', index=False)

['\ufeff"Date"', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']


/tmp/ipython-input-1327919553.py:100: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Tweet'].replace('', np.nan, inplace=True)
/tmp/ipython-input-1327919553.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [15]:
import csv

with open('S&P 500 Historical Data (1).csv', 'r', newline='') as SP_csv:
    reader = csv.DictReader(SP_csv)

    print(reader.fieldnames)

['\ufeff"Date"', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']


In [ ]:
'''

import tensorflow as tf

import numpy as np
import os
import time

path_to_file = 'trumpTweets.txt'

# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Split text charecters into tokens before conversion
chars = tf.strings.unicode_split(text, input_encoding='UTF-8')

# Convert tokens into nmeric IDs
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

ids = ids_from_chars(chars)

# To reverse

def text_from_ids(ids):
  chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)
print(text_from_ids(ids).numpy())

#Convert text vector into a stream of charecter indices
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
print(all_ids)

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

seq_length = 100

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

#for seq in sequences.take(1):
#print(chars_from_ids(seq))

#joining tokens into string

#for seq in sequences.take(5):
#  print(text_from_ids(seq).numpy())


def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list("Tensorflow"))
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

'''

'\n\nimport tensorflow as tf\n\nimport numpy as np\nimport os\nimport time\n\npath_to_file = \'trumpTweets.txt\'\n\n# Read, then decode for py2 compat.\ntext = open(path_to_file, \'rb\').read().decode(encoding=\'utf-8\')\n\n# Split text charecters into tokens before conversion\nchars = tf.strings.unicode_split(text, input_encoding=\'UTF-8\')\n\n# Convert tokens into nmeric IDs\nids_from_chars = tf.keras.layers.StringLookup(\n    vocabulary=list(vocab), mask_token=None)\n\nids = ids_from_chars(chars)\n\n# To reverse\n\ndef text_from_ids(ids):\n  chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)\n  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)\nprint(text_from_ids(ids).numpy())\n\n#Convert text vector into a stream of charecter indices\nall_ids = ids_from_chars(tf.strings.unicode_split(text, \'UTF-8\'))\nprint(all_ids)\n\nids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)\n\nfor ids in ids_dataset.